In [ ]:
# Install required packages for local reasoning
!pip install transformers torch accelerate --quiet

# Import libraries
import pandas as pd
import numpy as np
import torch
import warnings
import json
import os
from pathlib import Path
import time
from datetime import datetime

warnings.filterwarnings('ignore')

print("🚀 Fraud Detection Reasoning Environment Setup")
print("=" * 50)
print(f"✅ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  Using CPU - consider enabling GPU accelerator")

print("✅ Environment ready for local reasoning!")

# 🧠 Local Fraud Detection Reasoning on Kaggle

This notebook provides **local AI-powered reasoning** for fraud detection using Kaggle's GPU resources instead of paid APIs.



## 📊 Pipeline Flow
1. **Upload** your trained DistilBERT model
2. **Classify** texts into fraud categories  
3. **Generate reasoning** using local LM for non-legitimate classifications
4. **Download** results with explanations

## 🔧 Requirements
- Upload your `distilbert_model/` and `distilbert_tokenizer/` folders from previous training
- Enable GPU accelerator for faster inference
- No API keys needed!

In [ ]:
# Load Local Language Model for Reasoning (Free Alternative to APIs)
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

print("🧠 Loading Local Language Model for Reasoning...")
print("This replaces expensive API calls with free local inference!")

# Use a smaller, efficient model that works well on Kaggle's free tier
# Options: 'microsoft/DialoGPT-medium', 'gpt2', 'distilgpt2'
reasoning_model_name = "microsoft/DialoGPT-medium"  # Good balance of quality and speed

try:
    # Initialize reasoning pipeline
    reasoning_pipe = pipeline(
        "text-generation",
        model=reasoning_model_name,
        device=0 if torch.cuda.is_available() else -1,  # Use GPU if available
        do_sample=True,
        temperature=0.7,
        max_length=512,
        pad_token_id=50256  # Set pad token to avoid warnings
    )
    
    print(f"✅ Local reasoning model loaded: {reasoning_model_name}")
    print("💡 This model will generate explanations locally (no API costs!)")
    
    # Test the reasoning model
    test_prompt = "This text appears to be a scam because"
    test_response = reasoning_pipe(test_prompt, max_length=50, num_return_sequences=1)
    print("🧪 Model test successful!")
    
except Exception as e:
    print(f"⚠️  Error loading model: {e}")
    print("Falling back to simpler model...")
    # Fallback to smaller model
    reasoning_pipe = pipeline("text-generation", model="distilgpt2", device=0 if torch.cuda.is_available() else -1)

In [ ]:
# Load Your Trained DistilBERT Fraud Detection Model
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

print("📦 Loading Your Trained DistilBERT Model...")

# Update these paths to match where you uploaded your models
MODEL_PATH = '/kaggle/input/your-fraud-models/distilbert_model'  # Update this path
TOKENIZER_PATH = '/kaggle/input/your-fraud-models/distilbert_tokenizer'  # Update this path

# Class labels (must match your training - alphabetical order)
CLASS_LABELS = [
    'job_scam',
    'legitimate', 
    'phishing',
    'popup_scam',
    'refund_scam',
    'reward_scam',
    'sms_spam',
    'ssn_scam',
    'tech_support_scam'
]

try:
    # Load your trained model and tokenizer
    fraud_tokenizer = DistilBertTokenizer.from_pretrained(TOKENIZER_PATH)
    fraud_model = DistilBertForSequenceClassification.from_pretrained(MODEL_PATH)
    
    # Move to GPU for faster inference
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    fraud_model.to(device)
    fraud_model.eval()
    
    print(f"✅ DistilBERT model loaded successfully!")
    print(f"🎯 Device: {device}")
    print(f"📋 Classes: {len(CLASS_LABELS)} fraud types + legitimate")
    print(f"🏷️  Labels: {CLASS_LABELS}")
    
except FileNotFoundError:
    print("❌ Model files not found!")
    print("📁 Please upload your trained model files to Kaggle:")
    print("   1. Upload 'distilbert_model/' folder")
    print("   2. Upload 'distilbert_tokenizer/' folder") 
    print("   3. Update MODEL_PATH and TOKENIZER_PATH above")
    

# 🔧 Local Reasoning Engine Configuration

This section sets up the local reasoning engine that generates explanations for fraud classifications using the language model we loaded earlier.

In [ ]:
# Local Reasoning Engine Configuration
class LocalFraudReasoningEngine:
    """
    Local reasoning engine that generates explanations without API calls
    """
    
    def __init__(self, reasoning_pipeline):
        self.reasoning_pipe = reasoning_pipeline
        self.min_confidence = 0.5
        
        # Scam type descriptions for better reasoning
        self.scam_descriptions = {
            'phishing': {
                'description': 'Attempts to steal sensitive information like passwords, credit card numbers, or personal data',
                'indicators': ['urgent action required', 'verify account', 'click here', 'suspicious links', 'fake sender']
            },
            'popup_scam': {
                'description': 'Fake popup messages claiming virus infections or system issues',
                'indicators': ['virus detected', 'system error', 'immediate action', 'fake technical alerts']
            },
            'sms_spam': {
                'description': 'Unwanted promotional or fraudulent text messages',
                'indicators': ['unsolicited offers', 'prize claims', 'urgent responses', 'suspicious phone numbers']
            },
            'reward_scam': {
                'description': 'False promises of rewards, prizes, or free items',
                'indicators': ['congratulations', 'you have won', 'free gift', 'claim now', 'limited time']
            },
            'tech_support_scam': {
                'description': 'Fake technical support claiming to fix computer problems',
                'indicators': ['computer infected', 'microsoft support', 'remote access', 'technical issues']
            },
            'refund_scam': {
                'description': 'Fake refund notifications or requests for payment information',
                'indicators': ['refund available', 'payment failed', 'update payment', 'billing issue']
            },
            'ssn_scam': {
                'description': 'Attempts to steal Social Security Numbers or similar personal identifiers',
                'indicators': ['SSN verification', 'social security', 'identity verification', 'government agency']
            },
            'job_scam': {
                'description': 'Fake job offers or employment opportunities',
                'indicators': ['work from home', 'easy money', 'no experience required', 'guaranteed income']
            }
        }
        
        self.stats = {
            'total_processed': 0,
            'reasoning_generated': 0,
            'skipped_legitimate': 0,
            'skipped_low_confidence': 0
        }
        
    def should_generate_reasoning(self, predicted_label, confidence):
        """Determine if reasoning should be generated"""
        return predicted_label != 'legitimate' and confidence >= self.min_confidence
    
    def generate_local_reasoning(self, text, predicted_label, confidence, all_predictions):
        """Generate enhanced reasoning using local language model"""
        scam_info = self.scam_descriptions.get(predicted_label, {
            'description': 'Unknown scam type',
            'indicators': []
        })
        
        # Enhanced reasoning without relying on language model generation
        # Analyze text content directly
        text_lower = text.lower()
        detected_indicators = []
        
        # Check for specific indicators in the text
        for indicator in scam_info['indicators']:
            if any(word in text_lower for word in indicator.split()):
                detected_indicators.append(indicator)
        
        # Add common fraud patterns
        urgent_words = ['urgent', 'immediate', 'now', 'quickly', 'hurry', 'expires']
        if any(word in text_lower for word in urgent_words):
            detected_indicators.append('urgent language to pressure victims')
            
        money_words = ['$', 'money', 'prize', 'won', 'claim', 'free', 'gift']
        if any(word in text_lower for word in money_words):
            detected_indicators.append('financial incentives or rewards')
            
        action_words = ['click', 'call', 'text', 'visit', 'send', 'verify']
        if any(word in text_lower for word in action_words):
            detected_indicators.append('requests for immediate action')
            
        suspicious_elements = ['suspicious links', 'phone numbers', 'email addresses']
        if 'http' in text_lower or '@' in text_lower or any(char.isdigit() for char in text):
            detected_indicators.append('suspicious contact information')
        
        # Create comprehensive reasoning
        reasoning_parts = []
        reasoning_parts.append(f"This text was classified as {predicted_label} with {confidence:.1%} confidence.")
        reasoning_parts.append(f"\n{scam_info['description']}")
        
        if detected_indicators:
            reasoning_parts.append(f"\nKey fraud indicators detected:")
            for i, indicator in enumerate(detected_indicators[:4], 1):  # Limit to top 4
                reasoning_parts.append(f"• {indicator}")
        
        # Add context about why this is dangerous
        danger_context = {
            'phishing': 'This could lead to identity theft and financial loss.',
            'sms_spam': 'This could lead to unwanted charges and privacy violations.',
            'reward_scam': 'This could lead to financial scams and personal data theft.',
            'tech_support_scam': 'This could lead to remote access scams and financial fraud.',
            'job_scam': 'This could lead to advance fee fraud and identity theft.',
            'popup_scam': 'This could lead to malware installation and system compromise.',
            'refund_scam': 'This could lead to payment fraud and account takeover.',
            'ssn_scam': 'This could lead to identity theft and government impersonation fraud.'
        }
        
        if predicted_label in danger_context:
            reasoning_parts.append(f"\n⚠️ Risk: {danger_context[predicted_label]}")
        
        # Add confidence context
        if confidence > 0.9:
            reasoning_parts.append(f"\nHigh confidence ({confidence:.1%}) indicates strong fraud patterns.")
        elif confidence > 0.7:
            reasoning_parts.append(f"Moderate confidence ({confidence:.1%}) suggests probable fraud patterns.")
        
        return '\n'.join(reasoning_parts)

# Initialize the local reasoning engine
local_reasoning_engine = LocalFraudReasoningEngine(reasoning_pipe)
print("✅ Local reasoning engine initialized with enhanced analysis!")

In [ ]:
# Fraud Classification and Reasoning Functions
def classify_text(text, max_length=128):
    """Classify text using the loaded DistilBERT model"""
    if fraud_model is None:
        # Return sample prediction for demo
        return {
            'text': text,
            'predicted_label': 'phishing',
            'confidence': 0.92,
            'all_predictions': {
                'phishing': 0.92, 'legitimate': 0.04, 'popup_scam': 0.02,
                'reward_scam': 0.01, 'tech_support_scam': 0.01
            }
        }
    
    # Tokenize input
    encoding = fraud_tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    # Move to device
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Get predictions
    with torch.no_grad():
        outputs = fraud_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1).cpu().numpy()[0]
        predicted_class_id = np.argmax(probabilities)
    
    # Format results
    predicted_label = CLASS_LABELS[predicted_class_id]
    confidence = float(probabilities[predicted_class_id])
    
    all_predictions = {
        CLASS_LABELS[i]: float(probabilities[i]) 
        for i in range(len(CLASS_LABELS))
    }
    
    return {
        'text': text,
        'predicted_label': predicted_label,
        'confidence': confidence,
        'all_predictions': all_predictions
    }

def analyze_with_local_reasoning(text):
    """Complete analysis: classification + local reasoning"""
    # Step 1: Classify the text
    classification_result = classify_text(text)
    
    # Step 2: Generate local reasoning (only for non-legitimate classifications)
    if local_reasoning_engine.should_generate_reasoning(
        classification_result['predicted_label'], 
        classification_result['confidence']
    ):
        reasoning = local_reasoning_engine.generate_local_reasoning(
            text=classification_result['text'],
            predicted_label=classification_result['predicted_label'],
            confidence=classification_result['confidence'],
            all_predictions=classification_result['all_predictions']
        )
        
        local_reasoning_engine.stats['reasoning_generated'] += 1
        skip_reason = None
        reasoning_generated = True
    else:
        if classification_result['predicted_label'] == 'legitimate':
            skip_reason = 'legitimate_classification'
            local_reasoning_engine.stats['skipped_legitimate'] += 1
        else:
            skip_reason = f"low_confidence_{classification_result['confidence']:.2f}"
            local_reasoning_engine.stats['skipped_low_confidence'] += 1
        
        reasoning = None
        reasoning_generated = False
    
    local_reasoning_engine.stats['total_processed'] += 1
    
    return {
        **classification_result,
        'reasoning': reasoning,
        'reasoning_generated': reasoning_generated,
        'skip_reason': skip_reason,
        'timestamp': datetime.now().isoformat()
    }

def print_analysis_result(result):
    """Pretty print analysis result"""
    print("\n" + "="*80)
    print("🔍 LOCAL FRAUD DETECTION + REASONING ANALYSIS")
    print("="*80)
    
    print(f"\n📝 Original Text:")
    print(f"   {result['text']}")
    
    print(f"\n🎯 Classification:")
    print(f"   Label: {result['predicted_label']}")
    print(f"   Confidence: {result['confidence']:.2%}")
    
    print(f"\n📊 All Predictions:")
    for label, prob in sorted(result['all_predictions'].items(), key=lambda x: x[1], reverse=True):
        print(f"   {label}: {prob:.2%}")
    
    if result['reasoning_generated']:
        print(f"\n🧠 Local AI Reasoning:")
        print("   " + result['reasoning'].replace('\n', '\n   '))
    else:
        print(f"\n⏭️  Reasoning Skipped: {result['skip_reason']}")
    
    print("\n" + "="*80)

print("✅ Classification and reasoning functions ready!")
print("🚀 Ready to analyze texts with local AI reasoning!")

# 🧪 Sample Tests - Try Different Fraud Types

Let's test the local reasoning system with various types of fraudulent and legitimate messages.

In [ ]:
# Sample Test Cases for Different Fraud Types
sample_texts = [
    {
        'category': 'Phishing Attack',
        'text': "URGENT: Your PayPal account has been suspended due to suspicious activity. Click here immediately to verify your information and restore access: http://paypal-verification-secure.fraudsite.com"
    },
    {
        'category': 'Tech Support Scam', 
        'text': "WARNING: Your computer is infected with 5 viruses! Your files will be deleted in 24 hours. Call Microsoft Support immediately at 1-800-555-SCAM. Don't restart your computer or you'll lose everything!"
    },
    {
        'category': 'Reward Scam',
        'text': "🎉 CONGRATULATIONS! 🎉 You've been selected as our LUCKY WINNER for a $1000 Amazon gift card! You're one of only 3 winners today! Claim your prize now by clicking this link and entering your credit card info for verification. Hurry, expires in 1 hour!"
    },
    {
        'category': 'Job Scam',
        'text': "Amazing work from home opportunity! Earn $5000/week working just 2 hours per day! No experience required! Just send $99 registration fee and start earning today! Guaranteed income or money back!"
    },
    {
        'category': 'SMS Spam',
        'text': "FREE iPhone 15 Pro! You have been randomly selected as a winner. Text CLAIM to 12345 or visit bit.ly/freeiphone15winner to get your prize. Message and data rates may apply. Text STOP to opt out."
    },
    {
        'category': 'Legitimate Message',
        'text': "Hi Sarah, thank you for your order #12345. Your package has been shipped and will arrive within 3-5 business days. You can track your shipment using the tracking number provided in your confirmation email. Have a great day!"
    },
    {
        'category': 'SSN Scam',
        'text': "IMPORTANT NOTICE: Your Social Security Number has been suspended due to suspicious illegal activity. Call the SSA office immediately at 1-800-555-FAKE to verify your identity and reactivate your SSN. Failure to respond will result in arrest."
    }
]

print("🧪 Testing Local AI Reasoning on Sample Fraud Types")

for i, sample in enumerate(sample_texts):
    print(f"\n🎯 Test {i+1}: {sample['category']}")
    print("-" * 50)
    
    result = analyze_with_local_reasoning(sample['text'])
    print_analysis_result(result)

print(f"\n📊 Summary:")
print(f"Total Processed: {local_reasoning_engine.stats['total_processed']}")
print(f"Reasoning Generated: {local_reasoning_engine.stats['reasoning_generated']}")
print(f"Legitimate (Skipped): {local_reasoning_engine.stats['skipped_legitimate']}")


# 📝 Interactive Text Analysis

Enter your own text below to analyze with the local fraud detection + reasoning system.

In [ ]:
# Interactive Text Analysis
# Change the text below to analyze your own messages!

your_text = "Congratulations! You've won $1 million! Send your bank details to claim your prize!"

# Analyze your custom text
print("🔍 Analyzing Your Custom Text...")
custom_result = analyze_with_local_reasoning(your_text.strip())
print_analysis_result(custom_result)

# 📊 Batch Processing - Analyze Multiple Texts

Upload a CSV file or analyze multiple texts at once with local reasoning.

In [ ]:
# Batch Processing with Local Reasoning
def batch_analyze_texts(texts, save_results=True):
    """Analyze multiple texts and generate local reasoning"""
    results = []
    
    print(f"🔄 Processing {len(texts)} texts...")
    
    for i, text in enumerate(texts):
        if i % 5 == 0:  # Only print every 5th item to reduce clutter
            print(f"Progress: {i+1}/{len(texts)}")
        
        result = analyze_with_local_reasoning(text)
        results.append(result)
        
        # Small delay to avoid overwhelming the local model
        time.sleep(0.2)
    
    # Create summary DataFrame
    df_results = pd.DataFrame([
        {
            'text': r['text'][:100] + '...' if len(r['text']) > 100 else r['text'],
            'predicted_label': r['predicted_label'],
            'confidence': r['confidence'],
            'reasoning_generated': r['reasoning_generated'],
            'reasoning': r['reasoning'][:200] + '...' if r['reasoning'] and len(r['reasoning']) > 200 else r['reasoning'],
            'timestamp': r['timestamp']
        }
        for r in results
    ])
    
    if save_results:
        # Save results to CSV
        output_file = f'fraud_analysis_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
        df_results.to_csv(output_file, index=False)
        print(f"💾 Results saved to: {output_file}")
    
    return results, df_results

# Example batch processing
batch_texts = [
    "Your account will be closed unless you verify immediately!",
    "Hi John, thanks for the great meeting today. Let's follow up next week.",
    "You've won a free vacation! Call now to claim your prize!",
    "Your package has been delivered to your front door.",
    "URGENT: Your social security number has been compromised!"
]

print("📊 Batch Analysis with Local Reasoning")
batch_results, batch_df = batch_analyze_texts(batch_texts)

print("\n📈 Batch Analysis Summary:")
fraud_count = (batch_df['predicted_label'] != 'legitimate').sum()
reasoning_count = batch_df['reasoning_generated'].sum()
print(f"Fraud detected: {fraud_count}/{len(batch_df)}")
print(f"Reasoning generated: {reasoning_count}/{len(batch_df)}")

# Display sample results
display(batch_df.head())